<a href="https://colab.research.google.com/github/richardan01/Chat-with-your-data/blob/main/Look_ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Authenticate this session and choose project
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import ipywidgets as widgets
%reload_ext google.colab.data_table
auth.authenticate_user()
print("Authenticated.")

project_list = !gcloud projects list
project_ids = [row.split()[0] for row in project_list[1:]]
dummy_option = "(Change here if needed)"
options = project_ids.copy()
options.insert(0, dummy_option)

if 'fairprice-bigquery' in project_ids:
  project_id = "fairprice-bigquery"
  client = bigquery.Client(project=project_id)
else:
  project_id = project_ids[0]
  client = bigquery.Client(project=project_id)

print(f"Default project: `{project_id}` as of " + str(pd.Timestamp.now('Singapore')))

dropdown = widgets.Dropdown(
    options=options,
    value=dummy_option,
    description='Change? ',
)

def on_change(change):
  global project_id
  if change['name'] == 'value' and change['new'] in project_ids:
    project_id = change['new']
    client = bigquery.Client(project=project_id)
    print(f"Current project: `{project_id}` as of " + str(pd.Timestamp.now('Singapore')))

dropdown.observe(on_change)
display(dropdown)

Authenticated.
Default project: `gen-lang-client-0179548552` as of 2025-04-28 14:54:23.311537+08:00


Dropdown(description='Change? ', options=('(Change here if needed)', 'gen-lang-client-0179548552', 'gen-lang-c…

# 1.0 Sales Analysis

## **1.1 Sales Revenue by Category**

The table below shows total sales revenue of each product category available in Look Ecommerce

In [ ]:

%%bigquery --project $project_id
SELECT
  products.category,
  SUM(order_items.sale_price) AS total_sales,
  COUNT(DISTINCT order_items.user_id) AS total_customers
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
ON
  order_items.product_id = products.id
GROUP BY
  1
ORDER BY
  total_sales DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,category,total_sales,total_customers
0,Outerwear & Coats,1.347282e+06,8506
1,Jeans,1.251832e+06,11574
2,Sweaters,8.390357e+05,10214
3,Suits & Sport Coats,6.550920e+05,4830
4,Swim,6.531738e+05,10619
5,Fashion Hoodies & Sweatshirts,6.289732e+05,10737
6,Sleep & Lounge,5.540525e+05,10424
7,Shorts,5.181225e+05,10408
8,Tops & Tees,4.913341e+05,10918
9,Dresses,4.575463e+05,5137


- The most profitable product category is Outwear & Coats, followed by Jeans and Sweaters.
- The least profitable product categories are clothing sets, Jumpsuits & Rompers.

## **1.2 Sales Revenue percentage**

The table below shows revenue percentage in the ECommerce Store

In [ ]:
%%bigquery --project $project_id
-- total revenue by category and percentage
SELECT
  category,
  SUM(sale_price) AS total_revenue,
  ROUND((SUM(sale_price) / (
      SELECT
        SUM(sale_price)
      FROM
        `bigquery-public-data`.`thelook_ecommerce`.`order_items`)) * 100, 2) AS revenue_percentage
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
LEFT JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
ON
  order_items.product_id = products.id
GROUP BY
  category;

Query is running:   0%|          |

Downloading:   0%|          |

,category,total_revenue,revenue_percentage
0,Accessories,4.239708e+05,3.93
1,Intimates,4.532633e+05,4.20
2,Plus,1.649026e+05,1.53
3,Socks & Hosiery,6.266868e+04,0.58
4,Active,4.572503e+05,4.24
5,Maternity,2.601137e+05,2.41
6,Socks,1.235622e+05,1.14
7,Sleep & Lounge,5.496709e+05,5.09
8,Tops & Tees,4.952568e+05,4.59
9,Leggings,8.517522e+04,0.79


The top revenue driver is the **"Outwear and Coats"** category, which accounts for 12% of total sales. This indicates a strong product-market fit within the outerwear and coats segment.


## 1.3 Brand by revenue

The table below shows the revenue of brands sell in Look Ecommerce

In [ ]:
%%bigquery --project $project_id
SELECT
  p.brand AS brand,
  ROUND(SUM(oi.sale_price * o.num_of_item), 2) AS revenue,
  SUM(o.num_of_item) AS quantity
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS oi
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`orders` AS o
ON
  oi.order_id = o.order_id
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS p
ON
  oi.product_id = p.id
WHERE
  oi.status NOT IN ('Cancelled',
    'Returned')
GROUP BY
  1
ORDER BY
  revenue DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,brand,revenue,quantity
0,Calvin Klein,280062.82,4432
1,Diesel,272358.03,1999
2,True Religion,266291.86,1287
3,7 For All Mankind,240144.41,1522
4,Carhartt,239760.27,3536
...,...,...,...
2736,Luzy's Storage Place,12.95,1
2737,Wayfayrer,12.00,8
2738,Extenders,5.25,3
2739,Made in USA,3.92,8


**Analysis**
1. Calvin Klein brought in the most revenue.
2. Allegra K had the most sold quantity, double the amount sold by Calvin Klein.
3. Marshal Brand brought in the least revenue.

## 1.4 Sales Growth Over year
The table below shows the times series for sales growth over time


In [ ]:
# @title
%%bigquery --project $project_id
SELECT
  order_year,
  total_sales,
  ROUND((total_sales - LAG(total_sales, 1, 0) OVER (ORDER BY order_year)) * 100.0 / LAG(total_sales, 1, 1) OVER (ORDER BY order_year), 2) AS yoy_growth_percentage
FROM (
  SELECT
    EXTRACT(YEAR
    FROM
      DATE(delivered_at)) AS order_year,
    SUM(sale_price) AS total_sales
  FROM
    `bigquery-public-data`.`thelook_ecommerce`.`order_items`
  WHERE
    delivered_at IS NOT NULL
    AND returned_at IS NULL
    AND status = 'Complete'
  GROUP BY
    1 );

Query is running:   0%|          |

Downloading:   0%|          |

,order_year,total_sales,yoy_growth_percentage
0,2022,343255.480353,57.34
1,2021,218165.210411,95.75
2,2020,111449.440130,195.54
3,2025,536196.800545,-42.97
4,2019,37710.850076,3771085.01
5,2024,940173.541047,74.45
6,2023,538934.420719,57.01


**Analysis**

1. Big Y0Y% growth between 2019 to 2020
2. There is a decline bettween 2021 to 2022 but revenue start to increase back from 2023 to 2024



# **2.0 Profitable Analysis**


## 2.1 Overall Profitability over time

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT(YEAR
  FROM
    created_at) AS year,
  round(SUM(sale_price)) AS total_revenue,
  round(SUM(cost)) AS total_cost,
  round(SUM(sale_price - cost)) AS gross_profit
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
LEFT JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
ON
  order_items.product_id = products.id
GROUP BY
  1;

Query is running:   0%|          |

Downloading:   0%|          |

,year,total_revenue,total_cost,gross_profit
0,2025,2174242.0,1046205.0,1128037.0
1,2022,1416903.0,680396.0,736507.0
2,2024,3656191.0,1760857.0,1895334.0
3,2023,2149351.0,1032352.0,1116998.0
4,2021,856351.0,411692.0,444659.0
5,2020,458999.0,221013.0,237986.0
6,2019,134955.0,65062.0,69892.0


Operating profit trippled in 2020 due to increased revenue and cost efficiency.

## 2.2 **Profit margin by product category**


In [ ]:
%%bigquery --project $project_id
SELECT
  products.category,
  SUM(order_items.sale_price) AS total_revenue,
  SUM(order_items.sale_price - products.cost) AS total_profit,
  SUM(order_items.sale_price - products.cost) / SUM(order_items.sale_price) AS profit_margin,
  SUM(order_items.id) AS quantity_sold
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  products.id = order_items.product_id
GROUP BY
  1;



Query is running:   0%|          |

Downloading:   0%|          |

,category,total_revenue,total_profit,profit_margin,quantity_sold
0,Accessories,4.251903e+05,254767.683999,0.599185,907511688
1,Intimates,4.554635e+05,213096.735953,0.467868,1210008545
2,Plus,1.672646e+05,83580.431828,0.499690,388292714
3,Active,4.520963e+05,262418.437555,0.580448,814928040
4,Socks & Hosiery,6.346163e+04,37924.299667,0.597594,345330411
5,Maternity,2.713553e+05,151783.083437,0.559352,477015667
6,Sleep & Lounge,5.540525e+05,285693.257186,0.515643,1014363337
7,Socks,1.285261e+05,50956.530607,0.396468,558405224
8,Tops & Tees,4.913341e+05,216035.892655,0.439692,1077174599
9,Leggings,8.643598e+04,34528.503947,0.399469,280130578


This table shows the product profit margin

## 2.3 Recommendation for Profitability
**1. Increase Sales in High-Margin Products**


- From the profitability_improvement_analysis, we can identify which products have the highest gross margin percentages.
- Focusing marketing efforts and promotions on these products can drive higher overall profitability.


**2. Boost E-commerce Growth**

- Online channels are underperforming compared to in-store sales but typically offer higher profit margins due to lower operational costs (e.g., rent, staff).


**3. Reduce Operational Costs**

- Streamlining supply chain operations, improving inventory management, and negotiating better terms with suppliers can enhance gross margins.

# 3.0 Customer Behavior Analysis

## 3.1 Customer Segmentation based on age **group**

In [ ]:
%%bigquery --project $project_id
-- group it based on kids, teenagers, adults, elderly
SELECT
  CASE
    WHEN users.age BETWEEN 0 AND 12 THEN 'Kids'
    WHEN users.age BETWEEN 13
  AND 19 THEN 'Teenagers'
    WHEN users.age BETWEEN 20 AND 64 THEN 'Adults'
    ELSE 'Elderly'
END
  AS age_group,
  users.gender,
  COUNT(DISTINCT users.id) AS customer_count,
  SUM(orders.num_of_item) AS total_items_ordered,
  SUM(order_items.sale_price) AS total_revenue
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users` AS users
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`orders` AS orders
ON
  users.id = orders.user_id
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  orders.order_id = order_items.order_id
GROUP BY
  1,
  2;

Query is running:   0%|          |

Downloading:   0%|          |

,age_group,gender,customer_count,total_items_ordered,total_revenue
0,Adults,F,30561,131195,3.889533e+06
1,Elderly,F,3930,16824,4.977704e+05
2,Teenagers,F,4825,20753,6.259705e+05
3,Kids,F,689,3051,9.153246e+04
4,Adults,M,30555,129891,4.362759e+06
5,Elderly,M,4016,17206,5.676010e+05
6,Teenagers,M,4791,20047,6.655606e+05
7,Kids,M,657,2823,9.337649e+04


**Insight**
1. Most revenue is done by mostly Adults and Teenagers and kids the least
2. relatively low numbers in elderly and kids group

## 3.2 Customer Segmentation by Gender

In [ ]:
%%bigquery --project $project_id
-- customer segmentation by gender total revenue
SELECT
  users.gender,
  SUM(order_items.sale_price) AS total_revenue
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users` AS users
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`orders` AS orders
ON
  users.id = orders.user_id
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  orders.order_id = order_items.order_id
GROUP BY
  1;

Query is running:   0%|          |

Downloading:   0%|          |

,gender,total_revenue
0,M,5.680893e+06
1,F,5.111470e+06



Male customers generated more revenue and bought more products in quantity than female customers.

## 3.3 Customer Behavior Analysis : Total Customers by Country

In [ ]:
%%bigquery --project $project_id
SELECT
  users.country,
  SUM(order_items.sale_price) AS total_revenue
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users` AS users
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  users.id = order_items.user_id
GROUP BY
  1
ORDER BY
  total_revenue DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,country,total_revenue
0,China,3.655797e+06
1,United States,2.420322e+06
2,Brasil,1.568940e+06
3,South Korea,5.816685e+05
4,United Kingdom,5.041458e+05
5,France,5.014227e+05
6,Germany,4.482851e+05
7,Spain,4.432054e+05
8,Japan,2.704679e+05
9,Australia,2.333502e+05


## 3.4 Customer Behavior Analysis : Marketing Traffic Source

In [ ]:
%%bigquery --project $project_id
SELECT
  traffic_source,
  COUNT(*) AS total_customers
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users`
GROUP BY
  traffic_source;

Query is running:   0%|          |

Downloading:   0%|          |

,traffic_source,total_customers
0,Search,70216
1,Facebook,5956
2,Organic,14843
3,Email,4923
4,Display,4062


## 3.4 Recommendation to grow customer base
**1. Enhance Digital Marketing**
Focus on underperforming online channels by creating targeted ad campaigns, improving SEO, and enhancing the user experience.

**2. Implement Loyalty Programs:**
Reward high-value customers to encourage repeat purchases and increase customer lifetime value (CLV).

**3. Promotions During Off-Peak Periods: **
Implement marketing campaigns to drive traffic and sales during typically slower months.

# 4.0 Inventory Analysis

## 4.1 Inventory and overstock analysis

In [ ]:
%%bigquery --project $project_id
-- group it by prodcut category
SELECT
  t0.product_category,
  COUNT(t0.id) AS total_inventory,
  COUNT(CASE
      WHEN t0.sold_at IS NULL THEN t0.id
  END
    ) AS unsold_inventory,
  COUNT(CASE
      WHEN t0.sold_at IS NOT NULL THEN t0.id
  END
    ) AS sold_inventory
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`inventory_items` AS t0
GROUP BY
  1;

Query is running:   0%|          |

Downloading:   0%|          |

,product_category,total_inventory,unsold_inventory,sold_inventory
0,Accessories,26908,16920,9988
1,Active,24649,15552,9097
2,Blazers & Jackets,8672,5480,3192
3,Clothing Sets,557,352,205
4,Dresses,14430,9034,5396
5,Fashion Hoodies & Sweatshirts,31560,19844,11716
6,Intimates,36074,22732,13342
7,Jeans,33788,21230,12558
8,Jumpsuits & Rompers,2506,1585,921
9,Leggings,8828,5558,3270


**Analysis**

1. **Intimates** leads with around 36k total units. This indicates a major investment in this category but also a large proportion of unsold items.

2. **Jeans, Tops & Tees, Hoodies & Sweatshirts** also have high total inventory counts (~32k–34k).

3. **Outerwear & Coats and Accessories** appear to have relatively lower total inventory (in the mid-20k range), which could be strategic (due to higher cost per unit) or a result of stronger turnover.

# 5.0 Recommendation to improve the business

Based on the the  analysis across Sales, Profitability, Customer Behavior, and Inventory Management, here is some of the few recommendation to enchance business profitability:

**1. Enhance E-commerce Performance**

- Invest in SEO, digital marketing, user experience improvements, and a stronger mobile presence will attract more customers to the online channel.

- Implement online promotions and improving checkout processes can increase conversion rates and overall profitability.

**2. Optimize Inventory Management**

- From the inventory analysis, several products are overstocked while others frequently run out of stock.
- YOu can product bundle some product through promotion campaign or seasonal campaign to enchance profitabilty

**3. Implement Customer Loyalty Programs **

- Reward high-value customers identified from the segmentation analysis with  promotions, rewards, or  loyalty programs can increase customer retention and repeat purchases.

